# isee_machine & cycle_product
- isee_machine_NG + cycle-product
- 제품구분 통합
- 필요없는 row, cols 제거

In [15]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

## Merge

In [16]:
machine = pd.read_csv('./data/isee_machine_NG.csv')
product = pd.read_csv('./data/cycle-product.csv')

# 시간 type 변경
machine['creationTime'] = pd.to_datetime(machine['creationTime'],format = '%Y-%m-%d %H:%M')

In [17]:
mer_df = pd.merge(machine,product, how= 'left', left_on = 'CycleNum', right_on ='CycleNum')
mer_df.to_csv('./data/isee_match_NG.csv',index = False)

In [18]:
match_df = mer_df[(mer_df.Qua_Match_YN == 1) & (~mer_df.ProductNum.isnull())]

## Map data to a normal distribution

In [19]:
from sklearn.preprocessing import QuantileTransformer

In [20]:
analy_cols = ['CycleTime', 'VpPosition', 'VpTime','VpPressure', 'MaxInjectionPressure', 'InjectionEndPosition',
              'PlasticizingTime', 'PlasticsEndPosotion', 'ClampingTime', 'MouldOpeningTime', 'HotOilContoller', 'TankTemp', 'NozzleTemp',
              'Zone1', 'Zone2', 'Zone3', 'Zone4']

map_nomal_df = mer_df[analy_cols]

qt = QuantileTransformer(output_distribution='normal', random_state=2019)
map_nomal_df = qt.fit_transform(map_nomal_df)
mer_df[analy_cols] = map_nomal_df


In [21]:
# 정규화 한 후 전체 : 236696건
mer_df.to_csv('./data/isee_match_NG_MapNorm.csv',index = False)

In [22]:
#match_df.groupby(['ProductNum']).describe().T.to_excel('./result/isee_machine/describe_productNum_MapNorm.xlsx')

### cycle 처음과 끝의 시간차이가 1시간 미만 제거
- 초기, 중기, 말기가 모두 있는 cycle만 대상으로 하려고

In [23]:
isee_machine_operation = pd.DataFrame()

for c in match_df['CycleNum'].unique().tolist() :
    data = match_df[match_df.CycleNum == c].reset_index(drop = True)
    start_time = data['creationTime'][0]
    last_time = data['creationTime'][len(data)-1]
    if (last_time - start_time).total_seconds() <= 3600 : continue
    isee_machine_operation = pd.concat([isee_machine_operation,data],axis = 0, ignore_index = True)

In [24]:
print('총 데이터 건: ',isee_machine_operation.shape[0])
print('cycle 수: ',isee_machine_operation['CycleNum'].unique().shape[0])
print('NG라벨 불량 건: ',isee_machine_operation[isee_machine_operation.NG == 1].shape[0])
print('NG5라벨 불량 건: ',isee_machine_operation[isee_machine_operation.NG5 == 1].shape[0])

총 데이터 건:  185086
cycle 수:  109
NG라벨 불량 건:  7540
NG5라벨 불량 건:  27452


### 초기, 말기 데이터 삭제

In [25]:
isee_machine_operation = isee_machine_operation[isee_machine_operation.Operation_label == 1].reset_index(drop=True)

In [26]:
print('총 데이터 건: ',isee_machine_operation.shape[0])
print('cycle 수: ',isee_machine_operation['CycleNum'].unique().shape[0])
print('NG라벨 불량 건: ',isee_machine_operation[isee_machine_operation.NG == 1].shape[0])
print('NG5라벨 불량 건: ',isee_machine_operation[isee_machine_operation.NG5 == 1].shape[0])

총 데이터 건:  183201
cycle 수:  109
NG라벨 불량 건:  7416
NG5라벨 불량 건:  27102


In [27]:
isee_machine_operation.to_csv('./data/isee_machine_middle_NG_MapNorm.csv',index = False)

### 분산이 크거나 min_max 차이가 큰 컬럼 제외
- 직접 엑셀로 보기바람
- 'VpPosition','VpTime','InjectionEndPosition','PlasticsEndPosotion','TankTemp','NozzleTemp','Zone1','Zone2','Zone3','Zone4','CycleNum', 'Operation_label', 'True_label'

In [ ]:
describe = isee_machine_operation.describe().T

In [ ]:
describe['min_max_diff'] = [ describe['max'][i] - describe['min'][i] for i in range(len(describe))]

In [ ]:
mer_df.describe().T.to_csv('./result/isee_machine/all_describe_MapNorm.csv')

###  프로파일링 후 작성함

In [ ]:
# isee_machine_operation = isee_machine_operation[['VpPosition','VpTime','InjectionEndPosition','PlasticsEndPosotion','TankTemp','NozzleTemp'
#                                                 ,'Zone1','Zone2','Zone3','Zone4','CycleNum', 'Operation_label', 'NG','NG5',
#                                                 'OrderNum', 'ProductNum', 'ProductName', 'ModelNum']]

# isee_machine_operation.to_csv('./data/isee_machine_middle_removeclos_NG_MapNorm.csv',index = False)

# isee_machine_operation[isee_machine_operation.NG == 1].shape

# isee_machine_operation[isee_machine_operation.NG == 0].shape

### DB Insert
- 완성 부분 아님
#####  isee_machine, mer_df, isee_machine_operation 중 어떤것을 넣을지 잘생각해보기
- mer_df :  machine과 quality 통합시 통합되는 부분만 
- isee_machine : 원본에 생성컬럼 추가
- isee_machine_operation : 불필요한 row 제거

In [ ]:
# mer_df_tp = mer_df[['creationTime','YM','CycleNum','Operation_label','True_label','True_label5']].reset_index(drop = True)

# mer_df_tp['creationTime'] = mer_df_tp['creationTime'].astype(str)

# import pandas as pd
# import numpy as np
# import os
# import re
# import pymysql
# pymysql.converters.encoders[np.float64] = pymysql.converters.escape_float
# pymysql.converters.encoders[np.int64] = pymysql.converters.escape_int
# pymysql.converters.conversions = pymysql.converters.encoders.copy()
# pymysql.converters.conversions.update(pymysql.converters.decoders)

# conn = pymysql.connect(host='13.125.68.125',
#                          port=3306,
#                          user='rflow',
#                          passwd='agile~!@3',
#                          db='isee',
#                          charset='utf8',
#                          autocommit=True)

# with conn.cursor() as cursor:
#     try:
#         for idx in range(0,mer_df.shape[0]):
#             sql = """INSERT INTO isee.M_MACHINE_NewLabel (creationTime,
#                                                             YM,
#                                                             CycleNum,
#                                                             Operation_label,
#                                                             True_label,
#                                                             True_label5
#                                                             ) VALUES (%s, %s,%s, %s, %s,%s)"""

#             cursor.execute(sql, (mer_df_tp['creationTime'][idx],
#                                 mer_df_tp['YM'][idx],
#                                 mer_df_tp['CycleNum'][idx],
#                                  mer_df_tp['Operation_label'][idx],
#                                 mer_df_tp['True_label'][idx],
#                                 mer_df_tp['True_label5'][idx]))
#         conn.commit()
#         print(cursor.lastrowid)
#     except Exception as e :
#         conn.rollback() 
#         print(str(e))
#         print(" M_MACHINE_NewLabel db 저장 에러")
# conn.close()